## Setup

In [5]:
import os
import pandas as pd
import math
import numpy as np
from statistics import mean
# import nltk
import torch

from transformers import BertTokenizer, BertForSequenceClassification

c:\Users\Gabriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pré-processamento do texto

In [10]:
TRUE_PATH = r"dados\outros\True.csv"
FAKE_PATH = r"dados\outros\Fake.csv"
FINAL_PATH = r"dados\outros\Data.csv"

def get_processed_df(true_path: str, fake_path: str, final_path: str):
    """Método que:
    - Une as bases com noticias verdadeiras e falsas
    - Remove linhas com textos vazios
    - Coloca o texto em minúsculo
    - Remove linhas com texto e flag duplicadas
    - Se um texto está duplicado, mantém somente um

    true_path: Caminho da base com as noticias verdadeiras
    fake_path: Caminho da base com as noticias falsas
    final_path: Caminho do .CSV com a base processada
    """
    df_true = pd.read_csv(true_path)
    df_fake = pd.read_csv(fake_path)

    df_true['flag'] = 1
    df_fake['flag'] = 0

    df = pd.concat([df_true, df_fake])
    df = df[~df['text'].isnull()]

    df['text'] = df['text'].str.lower()

    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset='text', keep=False, inplace=True, ignore_index=True)

    df.to_csv(final_path, index=False)

In [11]:
get_processed_df(true_path=TRUE_PATH, fake_path=FAKE_PATH, final_path=FINAL_PATH)

In [12]:
import pandas as pd
df = pd.read_csv(r"dados\outros\Data.csv")

In [13]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33699 entries, 0 to 33698
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    33699 non-null  object
 1   text     33699 non-null  object
 2   subject  33699 non-null  object
 3   date     33699 non-null  object
 4   flag     33699 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


,title,text,subject,date,flag
0,"As U.S. budget fight looms, Republicans flip t...",washington (reuters) - the head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,washington (reuters) - transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,washington (reuters) - the special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,washington (reuters) - trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,seattle/washington (reuters) - president donal...,politicsNews,"December 29, 2017",1


In [14]:
df['tamanho_title'] = [len(i.split()) for i in df['title'].fillna('')]
df['tamanho_texto'] = [len(i.split()) for i in df['text'].fillna('')]

In [15]:
np.quantile(df['tamanho_title'], q=[.1, .2, .3, .4, .5, .6, .7, .8, .9])

array([ 8.,  9., 10., 10., 11., 11., 12., 13., 15.])

In [16]:
np.quantile(df['tamanho_texto'], q=[.1, .2, .3, .4, .5, .6, .7, .8, .9])

array([ 87., 164., 267., 329., 373., 411., 467., 557., 717.])

In [10]:
mean(df['tamanho_title'])

12.173316327945214

In [17]:
len(df[df['tamanho_texto'] < 500])/len(df)

0.7404077272322621

In [17]:
df = pd.concat([df_true, df_fake], ignore_index=True)

In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   flag    2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


,text,flag
0,The head of a conservative Republican faction ...,0
1,Transgender people will be allowed for the fir...,0
2,The special counsel investigation of links bet...,0
3,Trump campaign adviser George Papadopoulos tol...,0
4,President Donald Trump called on the U.S. Post...,0


In [37]:
df.groupby('flag').size()

flag
0    1000
1    1000
dtype: int64

In [36]:
df = df[~df['text'].isnull()]

In [42]:
df['text'] = df['text'].str.lower()

In [43]:
df['text']

0       the head of a conservative republican faction ...
1       transgender people will be allowed for the fir...
2       the special counsel investigation of links bet...
3       trump campaign adviser george papadopoulos tol...
4       president donald trump called on the u.s. post...
                              ...                        
1995    not long ago, rep. jason chaffetz (r-utah), sa...
1996    according to a new report, health insurance pr...
1997    the congressional budget office released their...
1998    donald trump was in a room full of irish repor...
1999    donald trump s war on the free press just ente...
Name: text, Length: 2000, dtype: object

In [2]:
# Rodar somente se o dataframe total nao tiver sido gerado
# get_processed_df(true_path=r"dados\DataSet_Misinfo_TRUE.csv", fake_path=r"dados\DataSet_Misinfo_FAKE.csv", final_path=os.path.join(os.path.split(true_path)[0], 'data.csv'))


Lendo os dados brutos

In [3]:
df = pd.read_csv(r"dados\data.csv")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78588 entries, 0 to 78587
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    78588 non-null  object
 1   flag    78588 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


,text,flag
0,The head of a conservative Republican faction ...,1
1,Transgender people will be allowed for the fir...,1
2,The special counsel investigation of links bet...,1
3,Trump campaign adviser George Papadopoulos tol...,1
4,President Donald Trump called on the U.S. Post...,1


In [40]:
df['text'].str.lower()

0       the head of a conservative republican faction ...
1       transgender people will be allowed for the fir...
2       the special counsel investigation of links bet...
3       trump campaign adviser george papadopoulos tol...
4       president donald trump called on the u.s. post...
                              ...                        
1995    not long ago, rep. jason chaffetz (r-utah), sa...
1996    according to a new report, health insurance pr...
1997    the congressional budget office released their...
1998    donald trump was in a room full of irish repor...
1999    donald trump s war on the free press just ente...
Name: text, Length: 2000, dtype: object

In [22]:
print(df['text'][78612])

The USA wants to divide Syria.

Great Britain, France, Germany and other EU and NATO states are to send their own troops and, above all, money to the areas of the north-east of the Euphrates, according to the will of the USA. This should secure the US's strategy, the division of Syria along the Euphrates, in the long term even after a planned partial withdrawal of US troops from the area.

&nbsp;


Obtendo a distribuição de palavras de cada texto

In [10]:
df['text'][1]

'Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District Court in the meantime,” the official said, speaking on condition of anonymity. In Septem

In [30]:
print(mean([len(i.split()) for i in df['text'].dropna()]))

12.268218397228122
540.8433455856856


In [35]:
df.groupby('flag').size()

label
0    35028
1    37106
dtype: int64

Importando o modelo pré-treinado

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Change num_labels according to your classification task

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [40]:
# Tokenize and preprocess input text
def preprocess_text(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=512,  # Adjust according to your input length requirements
        return_tensors='pt'
    )
    return inputs

In [47]:
# Example text for classification
text = "á à ã ç é ê õ ñ"
text_print = text.split()
text_print.append('[SEP]')
text_print.insert(0, '[CLS]')

# Preprocess input text
#inputs = preprocess_text(text)
inputs = tokenizer.encode_plus(text)
print(inputs['input_ids'])
print(tokenizer.convert_ids_to_tokens(inputs['input_ids']))
print(text_print)

[101, 1037, 1037, 1037, 1039, 1041, 1041, 1051, 1050, 102]
['[CLS]', 'a', 'a', 'a', 'c', 'e', 'e', 'o', 'n', '[SEP]']
['[CLS]', 'á', 'à', 'ã', 'ç', 'é', 'ê', 'õ', 'ñ', '[SEP]']


In [44]:
'[CLS]' + text.split()

TypeError: can only concatenate str (not "list") to str